In [1]:
%pylab inline
import ogr
import os
import shapely, shapely.wkt
import shapely_tools as st
import osm2dh
import utm_conversion as utm
from gdal_sample_points import gdal_sample_points
import numpy as np
import matplotlib.pyplot as plt
import pdb
import pyproj

Populating the interactive namespace from numpy and matplotlib


In [2]:
# TODO: multiLinestrings afvangen in filter_features
# TODO: dit verplaatsen naar osm2dh

def osm2channel(fn, check_fields, layer_index=1, # input 
                bbox=None, key='waterway', value='drain', # filter
                max_snap_dist = 1,  # geom changes
                depth=2., width=1., proftype=2, dtm_fn=None): # properties
    # filter features from OSM
    features = osm2dh.filter_features(fn, check_fields, bbox=bbox, key=key, value=value, layer_index=layer_index)
    
    # list of feature geometries
    geoms_edit = [ft['geom'] for ft in features]
    # snap and split lines
#     geoms_edit, index = st.snap_lines(geoms, max_dist=max_snap_dist, return_index=True)
    
    # temporary replacement
    index = range(len(features))
    
    objects = []
    for i, ift in enumerate(index):
        data = features[ift]
        data['geom'] = geoms_edit[i]
        # set default values
        data['proftype'] = proftype
        if not('depth' in data):
            data['depth'] = depth
        if not('width' in data):
            data['width'] = width
        ## TODO: make try except around this function
        # parse osm features and append to list
        o = osm2dh.Channel(bbox=bbox, dtm_fn=dtm_fn, **data)
        objects.append(o)
    return objects



In [3]:
# filter culverts, skip invalide fields and create gate objects
dataDir = r'd:\temp\CF1.2'
db_fn= os.path.join(dataDir, 'OSM', 'manzese_large-latest_v2.db')
dtm_fn = os.path.join(dataDir, 'DTM', 'manzese_DTM_0_5m_UTM37S_small.tif')

outDir = r'd:\temp\CF1.2\DFM_data'
pli_fn = os.path.join(outDir, 'channels', r'manzese_channels.pli')
dtm_xyz_fn = os.path.join(outDir, 'channels', r'manzese_channels_dtm_samples.xyz')
prof_xyz_fn = os.path.join(outDir, 'channels', r'manzese_channels_profloc.xyz')
profdef_fn = os.path.join(outDir, 'channels', r'manzese_channels_profdef.txt')

bnd_fn = os.path.join(outDir, 'boundaries', r'forcing.ext')
domain = shapely.geometry.Polygon([(524400, 9250200), (524400, 9247000), (526700, 9247000), (526700, 9250200)])

# purge any existing files
if os.path.exists(pli_fn):
    os.unlink(pli_fn)
if os.path.exists(dtm_xyz_fn):
    os.unlink(dtm_xyz_fn)
if os.path.exists(prof_xyz_fn):
    os.unlink(prof_xyz_fn)
if os.path.exists(profdef_fn):
    os.unlink(profdef_fn)

check_fields = {
    'width': float,
    'depth': float
}


# objects = osm2channel(db_fn, check_fields, key='waterway', value='drain', depth=5., width=10., proftype=2, layer_index=1, dtm_fn=dtm_fn)


In [4]:
keys = ['waterway', 'waterway', 'waterway', 'waterway']
values = ['ditch', 'stream', 'river', 'drain']
depths = [1., 1., 2., 0.5]
widths = [1., 1., 5., 0.5]
objects = []
for key, value, depth, width in zip(keys, values, depths, widths)[0:1]:
    print('Adding {:s} of type {:s} with depth {:.2f} and width {:.2f}'.format(key, value, depth, width))
    objects_part = osm2channel(db_fn,
                          check_fields=check_fields,
                          bbox=domain,
                          key=key,
                          value=value,
                          depth=depth,
                          width=width,
                          proftype=2,
                          layer_index=1,
                          dtm_fn=dtm_fn)
    objects += objects_part
# write to files
[o.to_pli(pli_fn, o._sname, o.feature, append=True) for o in objects]
[o.to_xyz(dtm_xyz_fn, o._geom, o.dtm_samples, append=True) for o in objects]
[o.to_xyz(prof_xyz_fn, o.profdef[n]['geometry'], [o._id], append=True) for o in objects for n in range(2)]
[o.to_profdef(profdef_fn, o.profdef[0]['properties'], append=True) for o in objects]
[o.to_bnd_cnds(bnd_fn, o.bnd_cnds[n], bnd_path=os.path.split(bnd_fn)[0], append=True) for o in objects for n in range(len(o.bnd_cnds))]



Adding waterway of type ditch with depth 1.00 and width 1.00
> d:\svn\openearthtools\python\applications\hydrotools\sandbox\osmmodelbuilding\osm2dh.py(223)to_bnd_cnds()
-> text_file.write(self.fmt_bnd_cnds(bnd_fn, bnd_cnd))
(Pdb) bnd_fn
'd:\\temp\\CF1.2\\DFM_data\\boundaries\\channel341763677_1_bnd.pli'
(Pdb) n
> d:\svn\openearthtools\python\applications\hydrotools\sandbox\osmmodelbuilding\osm2dh.py(224)to_bnd_cnds()
-> bnd_const_bnd_fn = os.path.join(bnd_path, _bnd_fn.rsplit('.', 1)[0] + '_0001.cmp')
(Pdb) q


BdbQuit: 

In [ ]:
o = objects[1]
len(o.bnd_cnds)
print os.path.split(bnd_fn)[0]

In [ ]:
def fmt_pli_header(name, geom, keys):
    pli_header_fmt = """{:s}
{:6d}{:6d}
""".format
    return pli_header_fmt(name, len(geom.xy[0]), 2+len(keys))
def fmt_pli(name, feature, keys):
    """
    Args:
        name:  name of feature to write
        feature: feature to write (1st and 2nd field goes to geometry)
        keys: keys to write to 3rd, 4th and xth fields

    Returns:

    """
    pli_row_fmt = str('{:15.3f}{:15.3f}' + '{:15.3f}'*len(keys) + '\n').format
    pli_header = fmt_pli_header(name, feature['geometry'], keys)
    pli_rows = ''
    for n, (x, y) in enumerate(zip(*feature['geometry'].xy)):
        vals = [float(feature['properties'][key]) for key in keys]
        pli_rows += pli_row_fmt(x, y, *vals)
    return pli_header + pli_rows

s = fmt_pli('some_geom', objects[1].feature, ['depth', 'id'])
print s

In [ ]:
#
fn_shp = r'd:\OneDrive\projects\1230843_Challenge_Fund\OSM\manzese_channels.shp'
fn_bnd_shp = r'd:\OneDrive\projects\1230843_Challenge_Fund\OSM\manzese_boundaries.shp'
# write new geometries to shapefile to check
from shapely.geometry import mapping, LineString
import fiona
from fiona import crs
epgs=32737  #  UTM37S

# Define a polygon feature geometry with one attribute
schema = {
    'geometry': 'LineString',
    'properties': {'id': 'int', 
                   'depth': 'float',
                   'width': 'float',
                  },
}

schema_bnd = {
    'geometry': 'LineString',
    'properties': {'id': 'int', 
                   'bnd_type': 'str',
                   'bnd_pli_fn': 'str',
                  },
}

# Write a new Shapefile
with fiona.open(fn_shp, 'w', 'ESRI Shapefile', schema, crs=crs.from_epsg(epgs)) as c:
    ## If there are multiple geometries, put the "for" loop here
    for o in objects:
        c.write({
            'geometry': mapping(o.geom_utm), # write line in utm
            'properties': {'id': int(o.osm_id), 
                           'depth':float(o.depth), 
                           'width':float(o.width),
#                            'proftype':float(o.proftype),
                           },
        })
# also write a shapefile for the boundary conditions
# Write a new Shapefile
with fiona.open(fn_bnd_shp, 'w', 'ESRI Shapefile', schema_bnd, crs=crs.from_epsg(epgs)) as c:
    ## If there are multiple geometries, put the "for" loop here
    for o in objects:
        for bnd_cnd in o.bnd_cnds:
            if bnd_cnd['geom'] is not None:
                c.write({
                    'geometry': mapping(bnd_cnd['geom']), # write boundary polyline in utm
                    'properties': {'id': int(o.osm_id), 
                                   'bnd_type':str(bnd_cnd['type']), 
                                   'bnd_pli_fn':str(bnd_cnd['bnd_fn']),
                                   },
                        })

In [ ]:
print str(bnd_cnd['bnd_fn'])